In [2]:
import os 
import sys 
from glob import glob 

from upaths import libd_rsdx_dpath 
from upaths import libb_prox_dpath
sys.path.append(libd_rsdx_dpath)
from utils import gwr_grid_downscaling

sys.path.append(libb_prox_dpath)
from rtransforms import raster_calc

from upaths import tls_spath,tls_bpath,mkd_spath,mkd_bpath,rng_spath,rng_bpath,gedi_egm_pattern
from rutils import adjust_for_geoid

def rm_aux_xml():
    path = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/DEMDOWNX/TILE12/*/*.aux.xml"
    files = glob(path)
    print(len(files))
    for fi in files:
        os.remove(fi)

    print('rm_aux_xml')

create viz checking points from vectors 
- locations : [x] n10e105, [x] n13e103, []s 
- clip rasters 
- viz and analysis 


In [6]:
#rm_aux_xml()
tilename = 'N10E105'
PROX_DPATH = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12"

In [8]:
gedi_egm_files = glob(gedi_egm_pattern); print(len(gedi_egm_files))
gedi_egm_file = [i for i in gedi_egm_files if tilename in i][0]
lidar_file = f"{PROX_DPATH}/{tilename}/{tilename}_multi_DTM_LiDAR.tif"
os.path.isfile(lidar_file)

17


True

In [9]:
# 1 and 1.5 appears to working for n10e105
# 1.8 to match # you can run optuna here with multiple values across all lidar to find the optimal value 
# in second iteration, use water mask river only from codem+esa to exclude those pixels 

In [ ]:
import rasterio
import numpy as np
import optuna
from sklearn.metrics import mean_squared_error, r2_score

def load_raster(filepath):
    """Load raster data and apply preprocessing (handle nodata, NaNs)."""
    with rasterio.open(filepath) as src:
        data = src.read(1).astype(np.float32)
        data[src.nodata] = np.nan
        data[(data >= 2000) | (data <= -30)] = np.nan
    return data

def optimize_adj_value(gedi_egm_file, reference_lidar_file):
    """Optimize the adjustment value using Optuna to minimize RMSE."""
    
    # Load rasters
    gedi_data = load_raster(gedi_egm_file)
    lidar_data = load_raster(reference_lidar_file)
    
    # Define optimization function
    def objective(trial):
        adj = trial.suggest_float("adj", 0.5, 2.0)  # Search range
        adjusted_gedi = gedi_data - adj
        
        valid_mask = ~np.isnan(adjusted_gedi) & ~np.isnan(lidar_data)
        rmse = mean_squared_error(lidar_data[valid_mask], adjusted_gedi[valid_mask], squared=False)
        
        return rmse  # Optuna minimizes this value
    
    # Run optimization
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)  # Adjust trials as needed
    
    best_adj = study.best_params["adj"]
    best_rmse = study.best_value
    
    # Compute R2 for best adj
    best_adjusted_gedi = gedi_data - best_adj
    valid_mask = ~np.isnan(best_adjusted_gedi) & ~np.isnan(lidar_data)
    best_r2 = r2_score(lidar_data[valid_mask], best_adjusted_gedi[valid_mask])

    return {"Best Adj": best_adj, "Best RMSE": best_rmse, "Best R2": best_r2}





/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import rasterio
import numpy as np
import optuna
from sklearn.metrics import mean_squared_error, r2_score

with rasterio.open(lwm_fpath) as src:
    ds = src.read(1)
    ndv = src.nodata

ndv  = 0.0
np.unique(ds) =  array([0, 1], dtype=uint8)
# add this mask into the analysis, and exclude where 0, only evaluarte whwere 1

def load_raster(filepath):
    """Load raster data and apply preprocessing (handle nodata, NaNs)."""
    with rasterio.open(filepath) as src:
        data = src.read(1).astype(np.float32)
        
        # Handle nodata values properly
        if src.nodata is not None:
            data[data == src.nodata] = np.nan
        
        # Apply value filters
        data[(data > 1000) | (data < -30)] = np.nan
    
    return data

def optimize_adj_value(gedi_egm_file, reference_lidar_file):
    """Optimize the adjustment value using Optuna to minimize RMSE."""
    
    # Load rasters
    gedi_data = load_raster(gedi_egm_file)
    lidar_data = load_raster(reference_lidar_file)
    
    # Define optimization function
    def objective(trial):
        adj = trial.suggest_float("adj", 0.5, 2.0)  # Search range
        adjusted_gedi = gedi_data - adj
        
        valid_mask = ~np.isnan(adjusted_gedi) & ~np.isnan(lidar_data)
        
        if np.sum(valid_mask) == 0:  # Avoid errors if no valid data points
            return float("inf")
        
        rmse = mean_squared_error(lidar_data[valid_mask], adjusted_gedi[valid_mask], squared=False)
        
        return rmse  # Optuna minimizes this value
    
    # Run optimization
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)  # Adjust trials as needed
    
    best_adj = study.best_params["adj"]
    best_rmse = study.best_value
    
    # Compute R2 for best adj
    best_adjusted_gedi = gedi_data - best_adj
    valid_mask = ~np.isnan(best_adjusted_gedi) & ~np.isnan(lidar_data)
    
    best_r2 = r2_score(lidar_data[valid_mask], best_adjusted_gedi[valid_mask]) if np.sum(valid_mask) > 0 else None

    return {"Best Adj": best_adj, "Best RMSE": best_rmse, "Best R2": best_r2}

# Example usage
# result = optimize_adj_value("gedi_egm.tif", "lidar_ref.tif")
# print(result)


In [ ]:
import rasterio
import numpy as np
import optuna
from sklearn.metrics import mean_squared_error, r2_score

def load_raster(filepath):
    """Load raster data and apply preprocessing (handle nodata, NaNs)."""
    with rasterio.open(filepath) as src:
        data = src.read(1).astype(np.float32)
        
        # Handle nodata values properly
        if src.nodata is not None:
            data[data == src.nodata] = np.nan
        
        # Apply value filters
        data[(data > 1000) | (data < -30)] = np.nan
    
    return data

def load_mask(mask_path):
    """Load binary mask where 1 = valid area, 0 = exclude."""
    with rasterio.open(mask_path) as src:
        mask = src.read(1).astype(np.uint8)
    
    return mask  # Returns 2D array with values 0 and 1

def optimize_adj_value(gedi_egm_file, reference_lidar_file, lwm_fpath):
    """Optimize the adjustment value using Optuna to minimize RMSE, considering a mask."""
    
    # Load rasters
    gedi_data = load_raster(gedi_egm_file)
    lidar_data = load_raster(reference_lidar_file)
    mask = load_mask(lwm_fpath)  # Binary mask (0: exclude, 1: include)
    
    # Ensure mask is binary (0 or 1)
    mask = (mask == 1)  # Converts it to a boolean array

    # Define optimization function
    def objective(trial):
        adj = trial.suggest_float("adj", 0.5, 2.0)  # Search range
        adjusted_gedi = gedi_data - adj
        
        # Create valid mask (valid values & included in mask)
        valid_mask = ~np.isnan(adjusted_gedi) & ~np.isnan(lidar_data) & mask
        
        if np.sum(valid_mask) == 0:  # Avoid errors if no valid data points
            return float("inf")
        
        rmse = mean_squared_error(lidar_data[valid_mask], adjusted_gedi[valid_mask], squared=False)
        
        return rmse  # Optuna minimizes this value
    
    # Run optimization
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)  # Adjust trials as needed
    
    best_adj = study.best_params["adj"]
    best_rmse = study.best_value
    
    # Compute R2 for best adj
    best_adjusted_gedi = gedi_data - best_adj
    valid_mask = ~np.isnan(best_adjusted_gedi) & ~np.isnan(lidar_data) & mask
    
    best_r2 = r2_score(lidar_data[valid_mask], best_adjusted_gedi[valid_mask]) if np.sum(valid_mask) > 0 else None

    return {"Best Adj": best_adj, "Best RMSE": best_rmse, "Best R2": best_r2}

# Example usage
# result = optimize_adj_value("gedi_egm.tif", "lidar_ref.tif", "mask.tif")
# print(result)


In [60]:
lwm_fpath = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N10E105/N10E105_LWM.tif"

In [67]:
import rasterio
import numpy as np
import optuna
from sklearn.metrics import mean_squared_error, r2_score

def load_raster(filepath):
    """Load raster data and apply preprocessing (handle nodata, NaNs)."""
    with rasterio.open(filepath) as src:
        data = src.read(1).astype(np.float32)
        
        # Handle nodata values properly
        if src.nodata is not None:
            data[data == src.nodata] = np.nan
        
        # Apply value filters
        data[(data > 1000) | (data < -30)] = np.nan
    
    return data

def load_mask(mask_path):
    """Load binary mask where 1 = valid area, 0 = exclude."""
    with rasterio.open(mask_path) as src:
        mask = src.read(1).astype(np.uint8)
    
    return mask  # Returns 2D array with values 0 and 1

def optimize_adj_value(gedi_egm_file, reference_lidar_file, lwm_fpath):
    """Optimize the adjustment value using Optuna to minimize RMSE, considering a mask."""
    
    # Load rasters
    gedi_data = load_raster(gedi_egm_file)
    lidar_data = load_raster(reference_lidar_file)
    mask = load_mask(lwm_fpath)  # Binary mask (0: exclude, 1: include)
    
    # Ensure mask is binary (0 or 1)
    mask = (mask == 1)  # Converts it to a boolean array

    # Define optimization function
    def objective(trial):
        adj = trial.suggest_float("adj", 0.8, 2.0)  # Search range
        adjusted_gedi = gedi_data - adj
        
        # Create valid mask (valid values & included in mask)
        valid_mask = ~np.isnan(adjusted_gedi) & ~np.isnan(lidar_data) & mask
        
        if np.sum(valid_mask) == 0:  # Avoid errors if no valid data points
            return float("inf")
        
        rmse = mean_squared_error(lidar_data[valid_mask], adjusted_gedi[valid_mask], squared=False)
        
        return rmse  # Optuna minimizes this value
    
    # Run optimization
    #study = optuna.create_study(direction="minimize")
    
    #study.optimize(objective, n_trials=20)  # Adjust trials as needed
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=50, timeout=300) # # Run for 50 trials or 5 minutes
    
    best_adj = study.best_params["adj"]
    best_rmse = study.best_value
    
    # Compute R2 for best adj
    best_adjusted_gedi = gedi_data - best_adj
    valid_mask = ~np.isnan(best_adjusted_gedi) & ~np.isnan(lidar_data) & mask
    
    best_r2 = r2_score(lidar_data[valid_mask], best_adjusted_gedi[valid_mask]) if np.sum(valid_mask) > 0 else None

    return {"Best Adj": best_adj, "Best RMSE": best_rmse, "Best R2": best_r2}

# Example usage
# result = optimize_adj_value("gedi_egm.tif", "lidar_ref.tif", "mask.tif")
# print(result)


In [68]:
result = optimize_adj_value(gedi_egm_file, lidar_file, lwm_fpath)

[I 2025-02-17 11:47:37,950] A new study created in memory with name: no-name-2f8dfaea-1c12-46e9-a1ac-3e820dec7953
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-17 11:47:38,994] Trial 0 finished with value: 7.105199337005615 and parameters: {'adj': 1.9331604072830286}. Best is trial 0 with value: 7.105199337005615.
/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-17 11:47:40,041] Trial 1 finished with value: 7.066188335418701 and parameters: {'adj': 1.4590923371162394}. Best is trial 1 with value: 7

In [69]:
result

{'Best Adj': 1.1236791950066192,
 'Best RMSE': 7.057697296142578,
 'Best R2': 0.2719513773918152}

In [ ]:
with rasterio.open(lwm_fpath) as src:
    ds = src.read(1)
    ndv = src.nodata

ndv  = 0.0
np.unique(ds) =  array([0, 1], dtype=uint8)
# add this mask into the analysis, and exclude where 0, only evaluarte whwere 1

In [65]:
ndv

0.0

In [63]:
np.unique(ds)

array([0, 1], dtype=uint8)

In [61]:
#result = optimize_adj_value(gedi_egm_file, lidar_file)

In [57]:
gedi_data = np.array([[10, 12], [15, np.nan]])
adj = 1.2  # Scalar from Optuna
adjusted_gedi = gedi_data - adj


In [ ]:
result # my objective functino should be the RMSEmap ?

{'Best Adj': 1.1618517886852733,
 'Best RMSE': 6.880164623260498,
 'Best R2': 0.271220862865448}

In [ ]:
result = optimize_adj_value(gedi_egm_file, lidar_file)

In [ ]:
i want to optimize adj value for this raster operations 

adj_list = [0.5,0.8,0.9,1,1.1,1.2, up to 2]
input raster is gedi_egm_file
operation is input_raster - adj_value (transform into 2d with same values)
get RMSE and R2 between gedi_adj_file and reference lidar_file
return a df with columns adj,RMSE,R2 


in reading raster , src.nodata set to null , anything above 1000 set to np.nan and less than -30 to np.nan too 

In [ ]:
adj = 1.5
gedi_adj_file = gedi_egm_file.replace('.tif', f'{adj}_adj.tif')
adjust_for_geoid(input_raster=gedi_egm_file, 
                 output_raster=gedi_adj_file, 
                 constant=adj, operation='subtract')

get RMSE and R2 between gedi_adj_file and reference lidar_file
i want to optim

##### ajust for geoid 

to do run HPO optimazation to find the best threshold in  [2MR]
- 1. TLS 
- 2. MKD 
- 3. RNG 
- 4. 1+2+3

Output raster '/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/DEMDOWNX/TILE12/N10E105/N10E105_GEDI03_dtm_EGM1.5_adj.tif' already exists. Operation not performed.


In [40]:
gedi_adj_clip_file = gedi_adj_file.replace('.tif', '_clip.tif')

#clip_raster_percentile(gedi_adj_file, gedi_adj_clip_file,lper=0.5,hper=100) # no need to clip heigh bound its water
# elevations are heihest in the water rivers and stuff 

In [30]:
# VERSION 1 WITHTOUT DSM 
# VERSION 2 WITH DSM

In [ ]:


# Example usage
# clip_raster_percentile("input.tif", "output.tif")


In [ ]:
# given a raster, i want to clip out the bottom 2 percentile, and top 80 percentile values, replace by nan
# after removing them save the raster 